In [1]:
from data.api import API
from data.solver_config import SolverConfig
from data.stock import Stock
from data.portfolio import Portfolio
from Utils.portfolio_solver import PortfolioSolver
from cvxopt import matrix, solvers
import numpy as np
import pandas as pd

stocks: dict[Stock, float] = {
    Stock('1', "Apple", "AAPL"): 0.2,
    Stock('2', "Microsoft", "MSFT"): 0.8
}

api = API('192.168.0.165')
config = SolverConfig()
solver = PortfolioSolver(stocks, config)


In [2]:
from datetime import date

stock_list = list(solver.stock_weights.items())
stock_count = len(stock_list)

price_data = api.get_price_history_for_tickers({ stock.ticker for stock in stocks.keys() }, date(2014, 1, 1), date(2015, 1, 1))
price_data_df = pd.DataFrame.from_dict(price_data, orient='index').T

YF.download() has changed argument auto_adjust default to True


In [4]:
# === Compute daily returns ===
var_returns = price_data_df.pct_change().dropna()
display(var_returns)

C:\Users\lafla\AppData\Local\Temp\ipykernel_12312\1616379198.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  var_returns = price_data_df.pct_change().dropna()


TypeError: unsupported operand type(s) for /: 'float' and 'decimal.Decimal'

In [ ]:
# === Compute expected returns (mu) ===
mu: np.ndarray = var_returns.mean().values.reshape(-1, 1)
print(mu)

In [ ]:
# === Compute covariance matrix (sigma) ===
sigma: np.ndarray = var_returns.cov().values
print(sigma)

In [ ]:
# === Normalize alpha scores ===
raw_scores = np.array([solver.stock_weights[stock].alpha_score for stock in stock_list])
normalized_scores = raw_scores / np.sum(raw_scores)
alpha_scores = normalized_scores.reshape(-1, 1)
print(alpha_scores)

In [ ]:
# === Optimization ===
p = matrix(solver.config.risk_aversion * sigma)
q = matrix(-mu - alpha_scores)

g = -np.eye(stock_count)
h = np.zeros(stock_count)

a = matrix(np.ones((1, stock_count)))
b = matrix(np.ones(1))

In [ ]:
sol = solvers.qp(p, q, G=matrix(g), h=matrix(h), A=a, b=b)
weights = np.array(sol['x']).flatten()

In [ ]:
# === Build Portfolio ===
metadata = {
    stock: Portfolio.StockMetadata(weight, solver.stock_weights[stock].alpha_score)
    for stock, weight in zip(stock_list, weights)
}

df = pd.DataFrame({
    stock.ticker: {'Weight': md.weight, 'Alpha Score': md.alpha_score}
    for stock, md in metadata.items()
}).T  # Transpose for nicer layout

display(df)